In [24]:
#paths
PATH = '/kaggle/input/hms-harmful-brain-activity-classification/'
EEG_PATH = '/kaggle/input/hms-harmful-brain-activity-classification/train_eegs'
import torch

In [25]:
import pandas as pd
import numpy as np

# Load the data
train = pd.read_csv(PATH+'train.csv')
test = pd.read_csv(PATH+'test.csv')
sample_submission = pd.read_csv(PATH+'sample_submission.csv')

label_cols = train.columns[-6:]
print(f"Train cataframe shape is: {train.shape}")
print(f"Labels: {list(label_cols)}")
train


Train cataframe shape is: (106800, 15)
Labels: ['seizure_vote', 'lpd_vote', 'gpd_vote', 'lrda_vote', 'grda_vote', 'other_vote']


,eeg_id,eeg_sub_id,eeg_label_offset_seconds,spectrogram_id,spectrogram_sub_id,spectrogram_label_offset_seconds,label_id,patient_id,expert_consensus,seizure_vote,lpd_vote,gpd_vote,lrda_vote,grda_vote,other_vote
0,1628180742,0,0.0,353733,0,0.0,127492639,42516,Seizure,3,0,0,0,0,0
1,1628180742,1,6.0,353733,1,6.0,3887563113,42516,Seizure,3,0,0,0,0,0
2,1628180742,2,8.0,353733,2,8.0,1142670488,42516,Seizure,3,0,0,0,0,0
3,1628180742,3,18.0,353733,3,18.0,2718991173,42516,Seizure,3,0,0,0,0,0
4,1628180742,4,24.0,353733,4,24.0,3080632009,42516,Seizure,3,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106795,351917269,6,12.0,2147388374,6,12.0,4195677307,10351,LRDA,0,0,0,3,0,0
106796,351917269,7,14.0,2147388374,7,14.0,290896675,10351,LRDA,0,0,0,3,0,0
106797,351917269,8,16.0,2147388374,8,16.0,461435451,10351,LRDA,0,0,0,3,0,0
106798,351917269,9,18.0,2147388374,9,18.0,3786213131,10351,LRDA,0,0,0,3,0,0


In [26]:
# Calculate the total votes for each `eeg_id` and `eeg_sub_id`
total_votes = train[label_cols].sum(axis=1)
# Loop over each classification and calculate the probability
for col in label_cols:
    train[col] = train[col] / total_votes

In [44]:
!pip install esig
!pip install signatory

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 680.9 kB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for signatory: filename=signatory-1.2.6.1.9.0-cp310-cp310-linux_x86_64.whl size=237935 sha256=18cd44d3dc78bd34eb90bc28b3feeeda73b604225f07734da9545c375e41dc22
  Stored in directory: /root/.cache/pip/wheels/71/b4/17/46d769da4808e9f83f9790a2b805f81f43ececc2c02f5b1e62
Successfully built signatory


In [72]:
import torch
from torch.utils.data import Dataset, DataLoader
import esig
import signatory
import pyarrow.parquet as pq

signature_depth = 2
siganture_length = sum(20**i for i in range(1,signature_depth+1))
print(siganture_length)


EEG_PATH = '/kaggle/input/hms-harmful-brain-activity-classification/train_eegs'

# class EEGDataset(Dataset):
#     def __init__(self, train_df):
#         self.train_df = train_df

#     def __len__(self):
#         return len(self.train_df['eeg_id'].unique())

#     def __getitem__(self, idx):
#         eeg_id = self.train_df['eeg_id'].unique()[idx]
#         eeg = torch.tensor(pd.read_parquet(EEG_PATH+f'/{eeg_id.item()}.parquet').values, device = 'cuda')[:10000,:]
#         eeg = (eeg - eeg.mean(axis=0))/eeg.std(axis=0)
#         #sig = torch.tensor(signatory.signature(eeg, signature_depth), dtype = torch.float32, device = 'cuda')
#         target = self.train_df.loc[(self.train_df['eeg_id'] == eeg_id) & (self.train_df['eeg_sub_id'] == 0), label_cols].values[0]
#         target = torch.tensor(target, dtype=torch.float32, device = 'cuda')
#         return eeg, target

class EEGDataset(Dataset):
    def __init__(self, train_df):
        self.train_df = train_df

    def __len__(self):
        return len(self.train_df['eeg_id'].unique())

    def __getitem__(self, idx):
        eeg_id = self.train_df['eeg_id'].unique()[idx]
        table = pq.read_table(EEG_PATH+f'/{eeg_id.item()}.parquet')
        eeg = torch.tensor(table.to_pandas().iloc[:10000].values, dtype=torch.float32, device = 'cuda')
        eeg = (eeg - eeg.mean(axis=0))/eeg.std(axis=0)
        #sig = torch.tensor(signatory.signature(eeg, signature_depth), dtype = torch.float32, device = 'cuda')
        target = self.train_df.loc[(self.train_df['eeg_id'] == eeg_id) & (self.train_df['eeg_sub_id'] == 0), label_cols].values[0]
        target = torch.tensor(target, dtype=torch.float32, device = 'cuda')
        return eeg, target

# Assuming you have the required variables defined
dataset = EEGDataset(train)
dataloader = DataLoader(dataset, batch_size=128, shuffle=True)


420


In [64]:
import torch.nn as nn
from torch.nn import functional as F

class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(siganture_length, 1000)
        self.fc2 = nn.Linear(1000, 100)
        self.fc3 = nn.Linear(100, 6)

    def forward(self, x):
        #standardize the input
        # x = (x - x.mean(axis=1)) / x.std(axis = 1)
        # x = torch.tensor(esig.stream2sig(x, signature_depth))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        x = F.softmax(x, dim=1)
        return x

In [47]:
torch.__version__
torch.device('cuda')
print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.cuda.current_device())
print(torch.cuda.get_device_name(0))
torch.device('cuda')

True
2
0
Tesla T4


device(type='cuda')

In [73]:
from tqdm import tqdm
num_epochs = 5
learning_rate = 0.001
nnmodel = MLP()
nnmodel.to('cuda')
criterion = nn.KLDivLoss(reduction="batchmean")
optimizer = torch.optim.Adam(nnmodel.parameters(), lr=learning_rate)
nnmodel.train()
for epoch in range(num_epochs):
    for eeg, targets in tqdm(dataloader):
        # Forward pass
        
        sig = signatory.signature(eeg, signature_depth)
        outputs = nnmodel(sig)
        targets = targets.to(outputs.dtype)  # Convert targets to the same data type as outputs
        loss = criterion(outputs, targets)
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')



  5%|▌         | 7/134 [00:14<04:14,  2.01s/it]


KeyboardInterrupt: 

In [ ]:
nnmodel

In [67]:
class LinearModel(nn.Module):
    def __init__(self):
        super(LinearModel, self).__init__()
        self.fc1 = nn.Linear(siganture_length, 1000)
        self.fc2 = nn.Linear(1000, 6)

    def forward(self, x):
        x = self.fc1(x)
        x = self.fc2(x)
        x = F.softmax(x, dim=1)
        return x



In [68]:
from tqdm import tqdm
num_epochs = 5
learning_rate = 0.001
linearmodel = LinearModel()
criterion = nn.KLDivLoss(reduction="batchmean")
optimizer = torch.optim.Adam(linearmodel.parameters(), lr=learning_rate)
linearmodel.train()
for epoch in range(num_epochs):
    for eeg, targets in tqdm(dataloader):
        # Forward pass
        outputs = linearmodel(eeg)
        targets = targets.to(outputs.dtype)  # Convert targets to the same data type as outputs
        loss = criterion(outputs, targets)
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

  4%|▍         | 6/134 [00:24<08:43,  4.09s/it]


KeyboardInterrupt: 

,time,LL_0.59,LL_0.78,LL_0.98,LL_1.17,LL_1.37,LL_1.56,LL_1.76,LL_1.95,LL_2.15,...,RP_18.16,RP_18.36,RP_18.55,RP_18.75,RP_18.95,RP_19.14,RP_19.34,RP_19.53,RP_19.73,RP_19.92
0,1,4.26,10.98,9.05,13.65,11.49,8.930000,18.840000,19.260000,19.240000,...,0.31,0.17,0.28,0.19,0.24,0.27,0.29,0.16,0.22,0.19
1,3,2.65,3.97,12.18,13.26,14.21,13.230000,9.650000,8.110000,11.280000,...,0.15,0.13,0.14,0.24,0.24,0.36,0.35,0.31,0.36,0.40
2,5,4.18,4.53,8.77,14.26,13.36,16.559999,19.219999,17.510000,22.650000,...,0.29,0.21,0.16,0.25,0.28,0.28,0.34,0.48,0.44,0.48
3,7,2.41,3.21,4.92,8.07,5.97,12.420000,10.820000,14.960000,21.809999,...,0.33,0.51,0.49,0.64,0.58,0.42,0.32,0.31,0.32,0.33
4,9,2.29,2.44,2.77,4.62,5.39,7.080000,9.840000,12.270000,14.410000,...,0.44,0.38,0.48,0.63,0.45,0.45,0.49,0.33,0.31,0.34
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,591,5.26,7.41,10.16,7.25,15.35,27.590000,28.650000,27.600000,30.690001,...,0.09,0.17,0.11,0.09,0.10,0.08,0.07,0.17,0.17,0.24
296,593,5.17,7.17,10.05,13.47,13.22,11.560000,29.790001,24.660000,23.379999,...,0.06,0.06,0.05,0.11,0.09,0.08,0.16,0.21,0.20,0.20
297,595,1.87,3.59,8.69,7.59,8.33,11.880000,12.690000,16.360001,24.240000,...,0.09,0.07,0.07,0.06,0.06,0.07,0.08,0.10,0.10,0.10
298,597,3.27,5.63,9.46,11.19,11.98,12.260000,9.240000,7.030000,8.660000,...,0.12,0.08,0.06,0.04,0.04,0.06,0.05,0.08,0.13,0.14


eeg_label_offset_seconds  spectrogram_id  \
eeg_id     eeg_sub_id                                             
1628180742 0                                0.0          353733   
           1                                6.0          353733   
           2                                8.0          353733   
           3                               18.0          353733   
           4                               24.0          353733   
...                                         ...             ...   
351917269  6                               12.0      2147388374   
           7                               14.0      2147388374   
           8                               16.0      2147388374   
           9                               18.0      2147388374   
           10                              20.0      2147388374   

                       spectrogram_sub_id  spectrogram_label_offset_seconds  \
eeg_id     eeg_sub_id                                                         
1628180742 0                            0                               0.0   
           1                            1                               6.0   
           2                            2                               8.0   
           3                            3                              18.0   
           4                            4                              24.0   
...                                   ...                               ...   
351917269  6                            6                              12.0   
           7                            7                              14.0   
           8                            8                              16.0   
           9                            9                              18.0   
           10                          10                              20.0   

                         label_id  patient_id expert_consensus  seizure_vote  \
eeg_id     eeg_sub_id                                                          
1628180742 0            127492639       42516          Seizure             3   
           1           3887563113       42516          Seizure             3   
           2           1142670488       42516          Seizure             3   
           3           2718991173       42516          Seizure             3   
           4           3080632009       42516          Seizure             3   
...                           ...         ...              ...           ...   
351917269  6           4195677307       10351             LRDA             0   
           7            290896675       10351             LRDA             0   
           8            461435451       10351             LRDA             0   
           9           3786213131       10351             LRDA             0   
           10          3642716176       10351             LRDA             0   

                       lpd_vote  gpd_vote  lrda_vote  grda_vote  other_vote  
eeg_id     eeg_sub_id                                                        
1628180742 0                  0         0          0          0           0  
           1                  0         0          0          0           0  
           2                  0         0          0          0           0  
           3                  0         0          0          0           0  
           4                  0         0          0          0           0  
...                         ...       ...        ...        ...         ...  
351917269  6                  0         0          3          0           0  
           7                  0         0          3          0           0  
           8                  0         0          3          0           0  
           9                  0         0          3          0           0  
           10                 0         0          3          0           0  

[106800 rows x 13 columns]

In [1]:
# import pandas as pd

# # Assume `metadata` is a DataFrame with the metadata

# EEG_PATH = 'hms-harmful-brain-activity-classification/train_eegs/'
# SPEC_PATH = 'hms-harmful-brain-activity-classification/train_spectrograms/'

# train = pd.read_csv('hms-harmful-brain-activity-classification/train.csv')


# # Set the index of the metadata for efficient querying
# train.set_index(['eeg_id', 'eeg_sub_id'], inplace=True)

# # Initialize a dictionary to store the EEG subsamples
# eeg_subsamples = {}

# # Loop over the unique EEG IDs and sub-IDs in the metadata
# for eeg_id, eeg_sub_id in train.index.unique():

#     # Load the EEG data from the parquet file
#     eeg_data = pd.read_parquet(EEG_PATH+f'{eeg_id}.parquet')

#     # Extract the start time and end time for this subsample from the metadata
#     start_time = train.loc[(eeg_id, eeg_sub_id), 'eeg_label_offset_seconds']
#     start_time = int(start_time)
#     end_time = start_time + 50  # Each subsample is 50 seconds long

#     # Extract the subsample from the EEG data
#     subsample = eeg_data.iloc[start_time*200:end_time*200]

#     # Store the subsample in the dictionary
#     eeg_subsamples[(eeg_id, eeg_sub_id)] = subsample

: 

In [11]:
# # Set the index of the metadata for efficient querying
# train = pd.read_csv('hms-harmful-brain-activity-classification/train.csv')

# train.set_index(['eeg_id', 'eeg_sub_id'], inplace=True)

# # Initialize a dictionary to store the EEG subsamples
# eeg_subsamples = {}

# # Get the unique EEG IDs
# unique_eeg_ids = train.reset_index()['eeg_id'].unique()

# # Loop over the unique EEG IDs
# for eeg_id in unique_eeg_ids:

#     # Get the unique sub-IDs for this EEG ID
#     unique_sub_ids = train.loc[eeg_id].index.unique()

#     # Loop over the unique sub-IDs
#     for eeg_sub_id in unique_sub_ids:

#         # Load the EEG data from the parquet file
#         eeg_data = pd.read_parquet(f'hms-harmful-brain-activity-classification/train_eegs/{eeg_id}.parquet')

#         # Extract the start time and end time for this subsample from the metadata
#         start_time = int(train.loc[(eeg_id, eeg_sub_id), 'eeg_label_offset_seconds'])
#         end_time = start_time + 50  # Each subsample is 50 seconds long

#         # Extract the subsample from the EEG data
#         subsample = eeg_data.iloc[start_time*200:end_time*200]

#         # Store the subsample in the dictionary
#         eeg_subsamples[(eeg_id, eeg_sub_id)] = subsample


: 